# Orthogonal Basis Decomposition
Decompose a PV tendency field into intensification (β), propagation (αx, αy), and deformation (γ) components.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pvtend.decomposition import compute_orthogonal_basis, project_field
from pvtend.plotting import plot_four_basis, plot_field_2d

## 1. Generate Synthetic PV Anomaly
Create a blocking-like negative PV anomaly and its gradients on a 40°×40° event-centred patch.

In [ ]:
# Event-centred relative coordinates (degrees)
x_rel = np.arange(-20, 21, 1.5)  # ~27 points
y_rel = np.arange(-20, 21, 1.5)
X, Y = np.meshgrid(x_rel, y_rel)

# PV anomaly: negative Gaussian (blocking signature)
sigma_x, sigma_y = 8.0, 6.0
q_prime = -2e-6 * np.exp(-X**2 / (2 * sigma_x**2) - Y**2 / (2 * sigma_y**2))

# PV gradients (centred differences)
dy_m = np.deg2rad(1.5) * 6.371e6
dx_m = dy_m * np.cos(np.deg2rad(50))  # ~ 50°N
dqdx = np.gradient(q_prime, dx_m, axis=1)  # ∂q/∂x
dqdy = np.gradient(q_prime, dy_m, axis=0)  # ∂q/∂y

print(f"Patch: {len(y_rel)}×{len(x_rel)} grid points")
print(f"q' range: [{q_prime.min():.2e}, {q_prime.max():.2e}] PVU")

## 2. Build Orthogonal Basis

In [ ]:
basis = compute_orthogonal_basis(
    pv_anom=q_prime,
    pv_dx=dqdx,
    pv_dy=dqdy,
    x_rel=x_rel,
    y_rel=y_rel,
    mask_negative=True,
    apply_smoothing=True,
    smoothing_deg=4.0,
    grid_spacing=1.5,
)

print("Basis norms:", {k: f"{v:.4f}" for k, v in basis.norms.items()})

## 3. Visualise the Four Basis Fields

In [ ]:
fig = plot_four_basis(
    basis.phi_int, basis.phi_dx, basis.phi_dy, basis.phi_def,
    x_rel, y_rel,
    suptitle="Orthogonal Basis for Synthetic Blocking Event",
)
plt.show()

## 4. Project Tendency onto Basis
Create a synthetic PV tendency (intensifying + eastward-moving pattern) and project.

In [ ]:
# Synthetic tendency: growth + eastward propagation
dqdt = 1e-11 * q_prime / q_prime.min()  # intensification
dqdt += 5e-12 * np.gradient(q_prime, dx_m, axis=1) * (-10)  # eastward advection

result = project_field(dqdt, basis)

print(f"β  (intensification) = {result['beta']:.3e} s⁻¹")
print(f"αx (zonal prop.)     = {result['ax']:.3e} m/s")
print(f"αy (merid. prop.)    = {result['ay']:.3e} m/s")
print(f"γ  (deformation)     = {result['gamma']:.3e} s⁻¹")
print(f"RMSE                 = {result['rmse']:.3e}")

## 5. Visualise Projection Components

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
components = [
    ("dq/dt (total)", dqdt),
    ("Intensification (β·Φ₁)", result["field_int"]),
    ("Propagation (α·Φ₂ + α·Φ₃)", result["field_prop"]),
    ("Residual", result["residual"]),
]
for ax, (title, field) in zip(axes.flat, components):
    vmax = np.nanmax(np.abs(field))
    if vmax < 1e-30:
        vmax = 1.0
    im = ax.imshow(field, origin="lower", cmap="coolwarm",
                    extent=[x_rel.min(), x_rel.max(), y_rel.min(), y_rel.max()],
                    vmin=-vmax, vmax=vmax)
    ax.set_title(title, fontsize=11)
    ax.set_xlabel("Δlon (deg)")
    ax.set_ylabel("Δlat (deg)")
    plt.colorbar(im, ax=ax, shrink=0.8)
plt.suptitle("PV Tendency Decomposition", fontsize=14)
plt.tight_layout()
plt.show()

The positive β confirms the blocking is intensifying, and the positive αx shows eastward propagation — consistent with the synthetic forcing we applied.